In [ ]:
EBS문장 ebs sentence list로 -> esl 한글로 전처리 -> 띄어쓰기 붙이기 -> 토크나이징

STT는..

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [2]:
from jamo import h2j, j2hcj, j2h

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical
from PreProcessing.Opentext import get_EBS

In [4]:
from soynlp.hangle import levenshtein
from soynlp.hangle import jamo_levenshtein
from soynlp.hangle import compose
from soynlp.hangle import decompose
from konlpy.tag import Kkma
import nltk
from nltk.tag import pos_tag
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import regexp_tokenize
from nltk.help import upenn_tagset
from string import punctuation
from nltk import Text
from konlpy.corpus import kolaw, kobill
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum
import pandas as pd
import numpy as np
import re
from PreProcessing.Dictionary import sd, ad, kopro
from PreProcessing.Opentext import get_EBS
from PreProcessing.Opentext import get_EBS_entered
from PreProcessing.Opentext import get_STT1_Google_entered
from PreProcessing.Opentext import get_STT1_Transcribe_entered
from collections import defaultdict
pd.set_option("display.max_rows", None, "display.max_columns", None)
from PreProcessing.find_common_part import find_common_part
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum

0
0
0
1
2
5
4
8


In [5]:
kopro

{0: '영',
 1: '일',
 2: '이',
 3: '삼',
 4: '사',
 5: '오',
 6: '육',
 7: '칠',
 8: '팔',
 9: '구',
 10: '십',
 11: '십일',
 12: '십이',
 13: '십삼',
 14: '십사',
 15: '십오',
 16: '십육',
 17: '십칠',
 18: '십팔',
 19: '십구',
 20: '이십',
 21: '이십일',
 22: '이십이',
 23: '이십삼',
 24: '이십사',
 25: '이십오',
 26: '이십육',
 27: '이십칠',
 28: '이십팔',
 29: '이십구',
 30: '삼십',
 31: '삼십일',
 32: '삼십이',
 33: '삼십삼',
 34: '삼십사',
 35: '삼십오',
 36: '삼십육',
 37: '삼십칠',
 38: '삼십팔',
 39: '삼십구',
 40: '사십',
 41: '사십일',
 42: '사십이',
 43: '사십삼',
 44: '사십사',
 45: '사십오',
 46: '사십육',
 47: '사십칠',
 48: '사십팔',
 49: '사십구',
 50: '오십',
 51: '오십일',
 52: '오십이',
 53: '오십삼',
 54: '오십사',
 55: '오십오',
 56: '오십육',
 57: '오십칠',
 58: '오십팔',
 59: '오십구',
 60: '육십',
 61: '육십일',
 62: '육십이',
 63: '육십삼',
 64: '육십사',
 65: '육십오',
 66: '육십육',
 67: '육십칠',
 68: '육십팔',
 69: '육십구',
 70: '칠십',
 71: '칠십일',
 72: '칠십이',
 73: '칠십삼',
 74: '칠십사',
 75: '칠십오',
 76: '칠십육',
 77: '칠십칠',
 78: '칠십팔',
 79: '칠십구',
 80: '팔십',
 81: '팔십일',
 82: '팔십이',
 83: '팔십삼',
 84: '팔십사',
 85: '팔십오',
 86: '팔십육',
 87:

In [6]:
kopro[1800000]

'백팔십만'

In [ ]:
def get_EBS(s, e):
    # 파일 이름 만들기 #################################################
    EBS_filenames = []
    for i in range(s, e+1):
        n = '{0:05d}'.format(i)
        EBS_fname = 'lec' + n + '_EBS.txt'
        EBS_filenames.append(EBS_fname)
    print(EBS_filenames)
    ####################################################
    
    # 파일 불러오기 ###################################################
    EBS_integrated = ''
    try:
        for _ in EBS_filenames:
            with open(r'data/' + _, 'r', encoding = 'utf8') as f:
                if EBS_filenames.index(_) == 0:
                    EBS_integrated += f.read() 
                else:
                    EBS_integrated += ' '+f.read()
    except:
        for _ in EBS_filenames:
            with open(r'data/' + _, 'r') as f:
                if EBS_filenames.index(_) == 0:
                    EBS_integrated += f.read() 
                else:
                    EBS_integrated += ' '+f.read()
    ####################################################
    
    return EBS_integrated

In [ ]:
EBS_entered = get_EBS(1, 26)

In [ ]:
# EBS 문장들 전처리
def EBS_preprocessing(EBS_entered):
    pat_math_symbol = re.compile(r'[().,‘’?!]') #의미없는부호 정규식
    pat_number = re.compile(r'\d+') #숫자 정규식
    esl = EBS_entered.splitlines() #문장단위로 리스트
    
    esl = [s.strip() for s in [pat_math_symbol.sub(' ', _) for _ in esl]]# punctuation 제거, 앞뒤 공백 제거
    
#     for _ in esl:
#         for i in esl.split():
#             all_dbdict.dbdict = defaultdict()
    
    #수학기호 한국어로 변환(EBS)
    esl_math_conversion = []
    for _ in esl:
        for ms in sd.keys():
            if ms in _:
                _ = _.replace(ms, sd[ms])
#                 if ms in dbdict['수학기호']:
#                     continue
#                 else:
#                     dbdict['수학기호'].append(ms)
#                     dbdict['한글발음'].append(sd[ms])
#                     dbdict['STT'].append('')
        esl_math_conversion.append(_)
        
    #숫자 한국어로 변환(EBS)
    esl_number_conversion = []
    for _ in esl_math_conversion:
        fl = re.findall(r'\d+', _)
        for n in fl:
            _ = _.replace(n, kopro[int(n)])
        esl_number_conversion.append(_)
        
    #알파벳 한국어로 변환(EBS)
    esl_alphabet_conversion = []
    for _ in esl_number_conversion:
        for a in ad.keys():
            if a in _:
                _ = _.replace(a, ad[a])
        esl_alphabet_conversion.append(_)
    EBS_pre = esl_alphabet_conversion
    return esl_alphabet_conversion

In [ ]:
EBS_ko = EBS_preprocessing(EBS_entered)

In [ ]:
EBS_ko = EBS_ko[0]

In [ ]:
EBS_ko

In [ ]:
def insert_dot(text, dot):
    
    pattern1 = re.compile(r'(ㅂㄴㅣㄷㅏ)$')   #ㅂ 니다
    pattern2 = re.compile(r'(.ㅔ|ㅏ|ㅓ|ㅐ|ㅗ|ㅜ)(ㅇㅛ)$') # ㅔ요, ㅏ요, ㅓ요
#     pattern3 = re.compile(r'(ㅆ)(ㅈ|ㅊ)(ㅛ)$') 
    pattern4 = re.compile(r'(ㅂ|ㅣ)(ㄴㅣㄲㅏ)$')  #ㅂ니까
    pattern5 = re.compile(r'(ㄴㄷㅔ)$')
    
    text_list = []
    
    _1 = 0
    _2 = 0
    _3 = 0
    _4 = 0
    _5 = 0
    _6 = 0
    
        
    for _ in text.split(' '): 
        
        new_ = j2hcj(h2j(_))
        if pattern1.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _1 += 1
            
            
        elif pattern2.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _2 += 1
            #list2.append(_)
        
#         elif pattern3.findall(new_):
#             text_list.append(_.replace(_, _+dot))
#             _3 += 1
#             list3.append(_)
        
        elif pattern4.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _4 += 1
            
        elif pattern5.findall(new_):
            text_list.append(_.replace(_,_+dot))
            _5 += 1
            
        else:
            text_list.append(_)
            _6 += 1
            #list6.append(_)
            
    print('pattern1 = {}, pattern2= {}, pattern3 = {}, pattern5 = {}, pattern6 = {}'.format(_1,_2,_3,_4,_5,_6)) 
    #print('pattern2 = ',list2)#'\n','list5 = ',list5)
    return text_list
    

In [ ]:
new_textE1 = ' '.join(insert_dot(EBS_ko,'.')).split('. ')

In [ ]:
new_textE1

In [ ]:
new_textE1 = [''.join(_.split()) for _ in new_textE1]

In [ ]:
new_textE1

In [ ]:
ma = Kkma().morphs

In [ ]:
corpus = new_textE1
law = Text(ma(corpus))


In [ ]:
law.tokens

In [ ]:
EBSlist_tokens = [' '.join(Text(ma(_)).tokens) for _ in new_textE1]

In [ ]:
EBSlist_tokens

In [ ]:
t = Tokenizer()
t.fit_on_texts(EBSlist_tokens)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

In [ ]:
sequences = list()

for line in EBSlist_tokens: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

In [ ]:
print(t.word_index)

In [ ]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

In [ ]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

In [ ]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

In [ ]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [ ]:
print(X[:3])

In [ ]:
print(y[:3]) # 레이블

In [ ]:
vocab_size

In [ ]:
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

In [ ]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence